# Table of Contents

1. [Creating a simple graph](#creating)
  *   [Using a snapshot representation](#sn)
  *  [Using an interval graph representation](#ig) 
2. [Visualization](#vizu)
3. [Conversion between snapshots and interval graphs](#conversion)
4. [Aggregation](#aggregation)

If tnerwork library is not installed, you need to install it, for instance using the following command

In [4]:
%%capture #avoid printing output
!pip install --upgrade git+https://github.com/Yquetzal/tnetwork.git

In [1]:
import tnetwork as tn

<a id='creating'></a>
# Creating simple dynamic graphs and accessing their properties
We will represent a graph with similar properties using snapshots and interval graphs
<a id='sn'></a>




##Using a snapshot representation



In [2]:
dg_sn = tn.DynGraphSN()

dg_sn.add_node_presence("a",1) #add node a in snapshot 1
dg_sn.add_nodes_presence_from(["a","b","c"],2) #add nodes a,b,c in snapshot 2
dg_sn.add_nodes_presence_from("d",[2,3]) #add node in snapshots 2 & 3

dg_sn.add_interaction("a","b",2) #link a and b in snapshot 2
dg_sn.add_interactions_from(("b","d"),[2,3]) #link b and d in snapshots 2 and 3

<a id='ig'></a>

## Using an interval graph representation.
Note the similarity between the functions used.

Both graphs are very similar if we consider that the snapshots of `dg_sn` have a duration of 1.

In [3]:
dg_ig = tn.DynGraphIG()

dg_ig.add_node_presence("a",(1,2)) #add node a from time 1 to 2 (1,2)
dg_ig.add_nodes_presence_from(["a","b","c"],(2,3)) # add ndoes a,b,c from 2 to 3
dg_ig.add_nodes_presence_from("d",(2,4)) #add node in snapshots 2 & 3

dg_ig.add_interaction("a","b",(2,3)) # link nodes a and b from 2 to 3 
dg_ig.add_interactions_from(("b","d"),(2,4))

## Accessing functions
Using accessing functions, we can check that both graphs are very similar

In [4]:
print(dg_sn.graph_at_time(2).edges)
print(dg_ig.graph_at_time(2).edges)
print(dg_sn.graph_at_time(3).edges)
print(dg_ig.graph_at_time(3).edges)

[('a', 'b'), ('b', 'd')]
[('a', 'b'), ('b', 'd')]
[('d', 'b')]
[('b', 'd')]


In [5]:
print(dg_sn.node_presence())
print(dg_ig.node_presence())

{'a': [1, 2], 'b': [2, 3], 'c': [2], 'd': [2, 3]}
{'a': [1,3[ , 'b': [2,4[ , 'c': [2,3[ , 'd': [2,4[ }


<a id='vizu'></a>

#Visualization
We can use a basic visualization to compare nodes presence of both representation. 

See the notebook on visualization to see more on that.

In [6]:
from bokeh.io import show, output_notebook

to_plot_SN = tn.plot_longitudinal(dg_sn,height=200)
to_plot_IG = tn.plot_longitudinal(dg_ig,height=200)


output_notebook()
show(to_plot_SN)
show(to_plot_IG)

/Users/cazabetremy/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:2591: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


Loading BokehJS ...

<a id='conversion'></a>

# Conversion between snapshots and interval graphs
We convert the snapshot representation into an interval graph representation, using a snapshot lenght of 1. 

We check that both graphs are now similar

In [7]:
converted_to_IG = dg_sn.to_DynGraphSG(sn_duration=1)
print(converted_to_IG.node_presence())
print(dg_ig.node_presence())

{'a': [1,3[ , 'b': [2,4[ , 'c': [2,3[ , 'd': [2,4[ }
{'a': [1,3[ , 'b': [2,4[ , 'c': [2,3[ , 'd': [2,4[ }


Reciprocally, we transform the interval graph into a snapshot representation and check the similarity

In [8]:
converted_to_SN = dg_ig.to_DynGraphSN(slices=1)
print(converted_to_SN.node_presence())
print(dg_sn.node_presence())

{'a': [1, 2], 'b': [2, 3], 'c': [2], 'd': [2, 3]}
{'a': [1, 2], 'b': [2, 3], 'c': [2], 'd': [2, 3]}


<a id='aggregation'></a>

# Aggregation
Sometimes, it is useful to study dynamic network with a lesser temporal granularity than the original data.

Several functions can be used to aggregate dynamic graphs, thus yielding snapshots covering larger periods.

Do exemplify this usage, we use a dataset from the sociopatterns project (http://www.sociopatterns.org) that can be loaded in a single command

In [9]:
sociopatterns = tn.DynGraphSN.graph_socioPatterns2012()

For this original network loaded as a snapshot representation, we print the number of snapshots and the first and last dates (the dataset covers 9 days, including a week-end with no activity)

In [10]:
from datetime import datetime
all_times = sociopatterns.snapshots_timesteps()
print("# snapshots:",len(all_times))
print("first date:",datetime.utcfromtimestamp(all_times[0])," laste date:",datetime.utcfromtimestamp(all_times[-1]))

# snapshots: 11273
first date: 2012-11-19 05:36:20  laste date: 2012-11-27 16:14:40


We then aggregate on fixed time periods using the `aggregate_time_period` function. Although there are several ways to call this function, the simplest one is using a string such as "day", "hour", "month", etc.
Note how the beginning of the first snapshot is now on midnight of the day on which the first observation was made

In [11]:
sociopatterns_Day = sociopatterns.aggregate_time_period("day")

In [12]:
all_times = sociopatterns_Day.snapshots_timesteps()
print("# snapshots:",len(all_times))
print("first date:",datetime.utcfromtimestamp(all_times[0])," laste date:",datetime.utcfromtimestamp(all_times[-1]))

# snapshots: 7
first date: 2012-11-19 00:00:00  laste date: 2012-11-27 00:00:00


Another way to aggregate is to use sliding windows. In this example, we use non-overlapping windows of one hour, but is possible to have other parameters, such as overlapping windows.
Note how, this time, the first snapshot starts exactly at the time of the first observation in the original data

In [13]:
sociopatterns_hour_window = sociopatterns.aggregate_sliding_window(bin_size=60*60)

In [14]:
all_times = sociopatterns_hour_window.snapshots_timesteps()
print("# snapshots:",len(all_times))
print("first date:",datetime.utcfromtimestamp(all_times[0])," laste date:",datetime.utcfromtimestamp(all_times[-1]))

# snapshots: 203
first date: 2012-11-19 05:36:20  laste date: 2012-11-27 15:36:20
